In [ ]:
import matplotlib.pyplot as plt
import pandas

# Ladataan data
Eduskuntavaalien 2019 anonymisoitu vaalikonedata.
Lähde: [YLE](https://yle.fi/uutiset/3-10725384)

In [ ]:
df = pandas.read_csv('Avoin_data_eduskuntavaalit_2019_valintatiedot.csv')
print('Data sisältää {rows} riviä ja {cols} kolumnia.'.format(rows=df.shape[0], cols=df.shape[1]))
df.head()

In [ ]:
df.groupby(['vaalipiiri'])['Euron ulkopuolella Suomi pärjäisi paremmin.'].mean()

Muutetaan kolumnien datatyypit numeroiksi.

In [ ]:
for col in df.columns:
    try:
        df[col] = df[col].apply(lambda x: None if x == '-' else float(x))
    except ValueError:
        continue

Tehdään pari nopeaa laskutoimitusta.

In [ ]:
df.groupby(['vaalipiiri'])['Euron ulkopuolella Suomi pärjäisi paremmin.'].mean()

In [ ]:
df.groupby(['puolue'])['Euron ulkopuolella Suomi pärjäisi paremmin.'].mean().sort_values(ascending=False)

Rajataan data vain yhteen vaalipiiriin.

In [ ]:
uusimaa = df[df['vaalipiiri'] == 'Uudenmaan vaalipiiri']
uusimaa.head()

In [ ]:
uusimaa.groupby(['puolue'])['Euron ulkopuolella Suomi pärjäisi paremmin.'].mean().sort_values(ascending=False)

In [ ]:
uusimaa.groupby(['puolue'])['Euron ulkopuolella Suomi pärjäisi paremmin.'].mean().sort_values(ascending=False) -\
    df.groupby(['puolue'])['Euron ulkopuolella Suomi pärjäisi paremmin.'].mean().sort_values(ascending=False)

In [ ]:
df.mean().head(10)

In [ ]:
groupcol = ['puolue', 'Valintatieto (0=ei valita, 1=valitaan, 2=varalla)']
df[[col for col in df.columns if 'Uusimaa' in col] + groupcol].groupby(groupcol).mean()

Rajataan data vain koko maata koskeviin kysymyksiin.

In [ ]:
datacols = df.columns[0:32].tolist() + ['Sukupuolesi', 'Koulutuksesi']
datacols = datacols[:2] + datacols[-2: ] + datacols[2:-2]
data = df[datacols]
data.head()

Tehdään yksinkertainen PCA visualisointia varten.

Aluksi, standardoidaan data (yksikkö varianssi, nolla keskiarvo).

In [ ]:
from sklearn.preprocessing import StandardScaler

features = data.columns[5:].tolist()
x = data.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

pcomp = pca.fit_transform(x)

Täytetään tyhjät arvot kolumnin keskiarvolla.

In [ ]:
data = data.fillna(data.mean())
x = data.loc[:, features].values
scaled = StandardScaler().fit_transform(x)
n_components = 4
pca = PCA(n_components=n_components)

pcomp = pca.fit_transform(scaled)
principaldf = pandas.DataFrame(data = pcomp, columns = ['pc{n}'.format(n=i+1) for i in range(n_components)])
print(pca.explained_variance_ratio_.sum())

In [ ]:
explained = pandas.DataFrame(pca.components_, columns=features,
                             index =['pc{n}'.format(n=i+1) for i in range(n_components)]).transpose()
explained.sort_values('pc1')

In [ ]:
principaldf.plot(kind='scatter', x='pc1', y='pc2', grid=True)
plt.show()

In [ ]:
pca_df = data[data.columns[:4]].merge(principaldf, left_index=True, right_index=True)
pca_df.head()

In [ ]:
import seaborn as sns
col = 'puolue'
# Unique category labels: 'D', 'F', 'G', ...
color_labels = pca_df[col].unique()

# List of RGB triplets
rgb_values = sns.color_palette("tab20", pca_df[col].nunique())

# Map label to RGB
color_map = dict(zip(color_labels, rgb_values))

ax = pca_df.plot(kind='scatter', x='pc1', y='pc2', c=df[col].map(color_map), grid=True, alpha=0.6)
grouped = pca_df.groupby(col)[['pc1', 'pc2']].mean()
grouped.plot(kind='scatter', ax=ax, x='pc1', y='pc2', c=grouped.index.map(color_map), grid=True, marker='X', s=100)
plt.show()

Kuva voisi olla interaktiivinen.

Aja komentorivillä: jupyter nbextension enable --py widgetsnbextension

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact_manual

from IPython.display import display

In [ ]:
plot_output = widgets.Output()
puolueet = pca_df['puolue'].unique().tolist()
puolueet.sort()
vaalipiirit = pca_df['vaalipiiri'].unique().tolist()
sukupuolet = pca_df['Sukupuolesi'].unique().tolist()
koulutukset = pca_df['Koulutuksesi'].unique().tolist()

puolue_widget = widgets.SelectMultiple(
    options=puolueet,
    description='Puolueet',
    disabled=False
)
vaalipiiri_widget = widgets.SelectMultiple(
    options=vaalipiirit,
    description='Vaalipiirit',
    disabled=False
)
sukupuoli_widget = widgets.SelectMultiple(
    options=sukupuolet,
    description='Sukupuoli',
    disabled=False
)
koulutus_widget = widgets.SelectMultiple(
    options=koulutukset,
    description='Koulutus',
    disabled=False
)
visualize_column_widget = widgets.Dropdown(
    options=['puolue', 'vaalipiiri', 'Sukupuolesi', 'Koulutuksesi'],
    default='puolue',
    description='Visualisoi kolumnin mukaan',
    disabled=False
)

def puolue_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,
                     bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, 
                     bounded_num.value)

def create_colormap(data, col):
    # Unique category labels: 'D', 'F', 'G', ...
    color_labels = data[col].unique()
    # List of RGB triplets
    rgb_values = sns.color_palette("tab20", data[col].nunique())
    # Map label to RGB
    color_map = dict(zip(color_labels, rgb_values))
    return color_map

    
def visualize_data(puolue, vaalipiiri, sukupuoli, koulutus, visualisoi):
    color_maps = {
        'puolue': create_colormap(pca_df, 'puolue'),
        'vaalipiiri': create_colormap(pca_df, 'vaalipiiri'),
        'Sukupuolesi': create_colormap(pca_df, 'Sukupuolesi'),
        'Koulutuksesi': create_colormap(pca_df, 'Koulutuksesi')
    }
    plot_output.clear_output()
    filtered = pca_df
    if puolue:
        filtered = filtered[filtered['puolue'].isin(puolue)]
    if vaalipiiri:
        filtered = filtered[filtered['vaalipiiri'].isin(vaalipiiri)]
    if sukupuoli:
        filtered = filtered[filtered['Sukupuolesi'].isin(sukupuoli)]
    if koulutus:
        filtered = filtered[filtered['Koulutuksesi'].isin(koulutus)]
    with plot_output:
        grouped = filtered.groupby(visualisoi)[['pc1', 'pc2']].mean()
        ax = filtered.plot(kind='scatter', x='pc1', y='pc2', c=filtered[visualisoi].map(color_maps[visualisoi]), grid=True,
                           legend=True, xlim=[-7, 7], ylim=[-4.5, 8])
        grouped.plot(kind='scatter', ax=ax, x='pc1', y='pc2', c=grouped.index.map(color_maps[visualisoi]), grid=True, marker='X', s=400)
        plt.show()

w = interactive(visualize_data, puolue=puolue_widget, vaalipiiri=vaalipiiri_widget,
                sukupuoli=sukupuoli_widget, koulutus=koulutus_widget,
                visualisoi=visualize_column_widget)
row1 = widgets.HBox([puolue_widget, vaalipiiri_widget])
row2 = widgets.HBox([sukupuoli_widget, koulutus_widget])
display(widgets.VBox([row1, row2, visualize_column_widget, plot_output]))